### 1. 加载公开词向量

In [13]:
from gensim.models import KeyedVectors

In [ ]:
model_path = 'data/sgns.weibo.word.bz2'
model = KeyedVectors.load_word2vec_format(model_path)

In [ ]:
# 查询词向量维度
print(model.vector_size)

# 查看词数
print('词数: ', len(model.index_to_key))

# 查询某个词的词向量
# print('词向量: ', model['自然语言处理'])

# 查看两个向量的相似度
similarity = model.similarity('自然语言处理', '机器学习')
print(f"自然语言处理 和 机器学习 的相似度: {similarity}")
similarity2 = model.similarity('地铁', '书本')
print(f"地铁 和 书本 的相似度: {similarity2}")
# 找出与某个词最相似的词
simmilar_words = model.most_similar(positive=['上班'], topn=10)
print("与 '上班' 最相似的词有:")
for word, score in simmilar_words:
    print(f"{word}: {score}")

300
词数:  195202
自然语言处理 和 机器学习 的相似度: 0.7483784556388855
地铁 和 书本 的相似度: 0.192057266831398
与 '上班' 最相似的词有:
赶点: 0.6254162192344666
下班: 0.6208966374397278
车限行: 0.5995197892189026
车限号: 0.5659466981887817
赶得: 0.5646229982376099
收假: 0.558693528175354
才醒: 0.5548821091651917
倒休: 0.5539935231208801
加班: 0.5507659912109375
练车: 0.5496417880058289


In [ ]:
# 男人-男孩+女孩=女人
model.most_similar(positive=['男人', '女孩'], negative=['男孩'], topn=5)

[('女人', 0.6578881740570068),
 ('女孩子', 0.515068531036377),
 ('女生', 0.45194485783576965),
 ('女人真', 0.4420627951622009),
 ('女人们', 0.43698593974113464)]

### 2. 训练自己的词向量

In [26]:
import pandas as pd
import jieba
from gensim.models import Word2Vec

In [19]:
df = pd.read_csv('data/online_shopping_10_cats.csv', encoding='utf-8').dropna()
display(df.head())

,cat,label,review
0,书籍,1,做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一...
1,书籍,1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...
2,书籍,1,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...
3,书籍,1,作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...
4,书籍,1,作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...


In [ ]:
# 分词 strip() 去除空格/换行符等
sentences = [[token for token in jieba.lcut(sentence) if token.strip() !=''] for sentence in df['review']]

[['做', '父母', '一定', '要', '有', '刘墉', '这样', '的', '心态', '，', '不断', '地', '学习', '，', '不断', '地', '进步', '，', '不断', '地', '给', '自己', '补充', '新鲜血液', '，', '让', '自己', '保持', '一颗', '年轻', '的', '心', '。', '我', '想', '，', '这', '是', '他', '能', '很', '好', '的', '和', '孩子', '沟通', '的', '一个', '重要', '因素', '。', '读', '刘墉', '的', '文章', '，', '总能', '让', '我', '看到', '一个', '快乐', '的', '平易近人', '的', '父亲', '，', '他', '始终', '站', '在', '和', '孩子', '同样', '的', '高度', '，', '给', '孩子', '创造', '着', '一个', '充满', '爱', '和', '自由', '的', '生活', '环境', '。', '很', '喜欢', '刘墉', '在', '字里行间', '流露出', '的', '做', '父母', '的', '那种', '小', '狡黠', '，', '让', '人', '总是', '忍俊不禁', '，', '父母', '和', '子女', '之间', '有时候', '也', '是', '一种', '战斗', '，', '武力', '争斗', '过于', '低级', '了', '，', '智力', '较量', '才', '更', '有', '趣味', '。', '所以', '，', '做', '父母', '的', '得', '加把劲', '了', '，', '老', '思想', '老', '观念', '注定', '会', '一败涂地', '，', '生命不息', '，', '学习', '不止', '。', '家庭教育', '，', '真的', '是', '乐在其中', '。'], ['作者', '真有', '英国人', '严谨', '的', '风格', '，', '提出', '观点', '、', '进行', '论述', '论证', '，', '尽管', '本人', '对', '物理

In [74]:
print(len(sentences))

62773


In [38]:
# 训练Word2Vec模型
model = Word2Vec(
    sentences,
    vector_size=100,
    window=5,
    min_count=2,
    sg=1,
    workers=4
)

In [42]:
model.wv.save_word2vec_format("data/word2vec.txt")

In [46]:
model = KeyedVectors.load_word2vec_format("data/word2vec.txt")

In [47]:
model.vector_size

100

### 3. 词向量应用

In [56]:
import torch.nn as nn
import torch

In [51]:
# 1. 加载词向量
wv = KeyedVectors.load_word2vec_format("data/word2vec.txt")

In [80]:
# 2. 处理OOV
unk_token = '<UNK>'
index2word = ['<UNK>'] + wv.index_to_key
print(index2word[0:5])
word2index = {word: index for index, word in enumerate(index2word)}

['<UNK>', '，', '的', '。', '了']


In [82]:
# 3. 准备词向量矩阵
num_embeddings = len(index2word)
print("词表大小:", num_embeddings)
embedding_dim = wv.vector_size
print("词向量维度:", embedding_dim)

embedding_matrix = torch.zeros(num_embeddings, embedding_dim)
for idx, word in enumerate(index2word):
    if word in wv:
        embedding_matrix[idx] = torch.tensor(wv[word])

print(wv.vectors.shape)
# embadding_matrix = torch.FloatTensor(index2word.vectors)

# 4. 创建Embedding层
embedding = nn.Embedding.from_pretrained(embedding_matrix)


词表大小: 34577
词向量维度: 100
(34576, 100)


In [87]:
# 5. 测试
text = "我喜欢乘坐地铁宇宙飞船"
tokens = jieba.lcut(text)  
input_ids = [word2index.get(token, word2index[unk_token]) for token in tokens]  
input_tensor = torch.tensor([input_ids]) # torch.Size([1, 4]) 包含了batch_size维度
embedding(input_tensor).shape 

torch.Size([1, 5, 100])